In [1]:
# library
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
document_embeddings = np.load("../local_data/_train_doc_embeddings.npy")
# document_embeddings = document_embeddings[:9000]

In [3]:
# Compute cosine similarity between document embeddings (Run time : 15 minutes)
similarity_matrix = cosine_similarity(document_embeddings)

In [24]:
# helper function to divdie work in pieces:
def spliterate(buf, chunk):
    for start in range(0, len(buf), chunk):
        yield buf[start:start + chunk]

In [37]:
# Create sparse matrix in chunks 
i = 0               # counter
threshold = 0.8     # threshold for similarlity value 

for partition in spliterate(similarity_matrix, 10000):
    i += 1
    matrix_with_zeros = np.where(partition < threshold, 0, partition)
    if i == 0:
        similarity_matrix_sparse = sparse.csr_matrix(matrix_with_zeros) 
        continue
    sparse_partition = sparse.csr_matrix(matrix_with_zeros) 
    similarity_matrix_sparse = sparse.vstack([similarity_matrix_sparse, sparse_partition])

print(similarity_matrix_sparse.shape)

In [ ]:
# Extract nonzero elements, row indices, and column indices
nonzero_data = similarity_matrix_sparse.data
row_indices, col_indices = similarity_matrix_sparse.nonzero()

# Create a DataFrame
graph_df_dupe = pd.DataFrame({'node_from': row_indices,
                   'node_to': col_indices,
                   'measure': nonzero_data})

# remove duplicates
graph_mask = graph_df_dupe["node_from"] < graph_df_dupe["node_to"]
graph_df = graph_df_dupe[graph_mask]

# print shape and few 
print(graph_df.shape)
graph_df.head()

(221393, 3)


,node_from,node_to,measure
0,0,90000,1.000000
1,1,90001,1.000000
2,2,159,0.985550
3,2,243,0.985159
4,2,610,0.986762


In [ ]:
# create csv for graph data
graph_df.to_csv('../local_data/_graph_data.csv', index=False)